## Hamza Mustafa
### Final Project

In [8]:
# Load Packages

!python --version
#%reset
import os
import math
import random
import numpy as np
import pandas as pd
import sklearn as sk

Python 3.11.1


In [9]:
#Directory Checking
os.listdir()
dir = r'D:\Docs\Code\Machine Learning\FinalProject\Manually_Checked_Global_Food_Dataset.csv'

#Save csv file as a dataframe
df = pd.read_csv(dir)

#Print dataframe
df.style

#Select desired columns
clean_df = df[['Country', 'Tons of food scarcity', 'Tons of surplus food']].copy()
#clean_df.style

clean_df['sub'] = clean_df['Tons of surplus food']-clean_df['Tons of food scarcity']
#clean_df.style

#clean_df.loc[clean_df['pclass'] == '1st', 'pclass'] = 1
#clean_df.loc[clean_df['pclass'] == '2nd', 'pclass'] = 2
#clean_df.loc[clean_df['pclass'] == '3rd', 'pclass'] = 3

#clean_df.loc[clean_df['sex'] == 'male', 'sex'] = 0
#clean_df.loc[clean_df['sex'] == 'female', 'sex'] = 1

#avg_age = clean_df['age'].mean(axis = 0)
#print(age_avg)

#Set nan values to average age
#clean_df['age'] = clean_df['age'].fillna(avg_age)

#clean_df.style

In [10]:

negative_values = clean_df[clean_df['sub'] < 0]['sub']

# Find maximum and minimum values among negative numbers
max_negative = negative_values.max()
min_negative = negative_values.min()

positive_values = clean_df[clean_df['sub'] >= 0]['sub']
max_positive = positive_values.max()
min_positive = positive_values.min()

a = 0.01
b = 10

c = -10
d = -0.01

clean_df['normalized_sub'] = clean_df['sub'].apply(lambda x: ((x - min_positive) / (max_positive - min_positive)) * (b - a) + a if x >= 0 else (x - min_negative) / (max_negative - min_negative) * (d - c) + c)

In [11]:
#clean_df.style

In [12]:
t1 = -1

print(math.floor(t1*10))

-10


In [13]:

#Select desired columns
norm_df = clean_df[['Country', 'normalized_sub']].copy()
#norm_df.style

norm_df['classification'] = norm_df['normalized_sub'].apply(lambda x: math.ceil(x) if x >= 0 else math.floor(x))

In [15]:
#Outputs
y = norm_df['classification'].copy()
print(y)

0     -3
1     -1
2     -1
3      1
4     -3
      ..
207    1
208   -1
209    1
210   -2
211   -2
Name: classification, Length: 212, dtype: int64


In [19]:
#Select desired columns
x_df = df[['GDP per Capita', 'Population', 'Infant Mortality Rate', 'Life Expectancy']].copy()

#Find average Infant Mortality Rate
avg_mr = x_df['Infant Mortality Rate'].mean(axis = 0)

#Set nan values to average rate
x_df['Infant Mortality Rate'] = x_df['Infant Mortality Rate'].fillna(avg_mr)

#Find average Life Expectancy
avg_le = x_df['Life Expectancy'].mean(axis = 0)

#Set nan values to average Life Life Expectancy
x_df['Life Expectancy'] = x_df['Life Expectancy'].fillna(avg_le)

x = x_df

In [21]:
#x_df.style

#Create training data using sklearn
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)